In [ ]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import wasserstein_distance
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy.ma as ma
from collections import defaultdict
import csv

from scipy.stats import ttest_1samp
from statistics import mean
from utils import *

In [ ]:
models = ['llama2-7b', 'llama2-13b', 'llama2-70b', 'llama2-70b-ift', 'llama2-7b-chat', 'llama2-13b-chat', 'llama2-70b-chat', 'gpt-3.5-turbo', 'gpt-3.5-turbo-instruct']  #these dont have perturbations yet
bias_types = ['acquiescence','response_order', 'odd_even', 'opinion_float', 'allow_forbid']
subset_bias_types = ['acquiescence-50','response_order-50', 'odd_even-50', 'opinion_float-50', 'allow_forbid']
perturbations = ['-key_typo', '-middle_random', '-letter_swap']


all_results = []

for model in models:
    print(model)

    for i in range(len(bias_types)):
            
        bias_type = bias_types[i]
        
        print(bias_type)
        
        orig_mean, orig_std, new_mean, new_std = get_entropies(model, bias_type)
        
        lst = [model, bias_type, orig_mean, orig_std, new_mean, new_std]
        
        for perturbation in perturbations:
                        
            if subset_bias_types[i] == 'opinion_float-50': #qustions are the same
                bias_type = 'odd_even'+perturbation
            else:
                bias_type = bias_types[i]+perturbation
        
            
            orig_mean, orig_std, new_mean, new_std = get_entropies(model, bias_type)
            lst.append(new_mean)
            lst.append(new_std)
        
        all_results.append(lst)
        
        

            
df = pd.DataFrame(all_results, columns = ['model', 'bias type', 'original mean', 'original std',\
                                          'bias mean', 'bias std',\
                                          'key typo mean', 'key typo std',\
                                          'middle random mean', 'middle random std',\
                                          'letter swap mean', 'letter swap std'])
df = df.round(4)

In [ ]:
import matplotlib.pyplot as plt 
from matplotlib.gridspec import SubplotSpec

def create_subtitle(fig: plt.Figure, grid: SubplotSpec, title: str):
    "Sign sets of subplots with title"
    row = fig.add_subplot(grid)
    # the '\n' is important
    row.set_title(f'{title}\n', fontweight='semibold')
    # hide subplot
    row.set_frame_on(False)
    row.axis('off')

clean_model_labels = ['Llama2-7b', 'Llama2-13b', 'Llama2-70b', 'Solar', 'Llama2-7b-chat', 'Llama2-13b-chat', 'Llama2-70b-chat', 'GPT 3.5 Turbo', 'GPT 3.5 Turbo Instruct']
models = ['llama2-7b', 'llama2-13b', 'llama2-70b', 'llama2-70b-ift', 'llama2-7b-chat', 'llama2-13b-chat', 'llama2-70b-chat', 'gpt-3.5-turbo', 'gpt-3.5-turbo-instruct']  #these dont have perturbations yet
bias_types = ['acquiescence','allow_forbid', 'response_order', 'opinion_float', 'odd_even']
clean_labels = ['Bias', 'Key Typo', 'Middle Random', 'Letter Swap']
clean_bias_labels = ['Acquiescence', 'Allow/forbid', 'Response Order', 'Opinion Float', 'Odd/even']

fig, axs = plt.subplots(len(models),len(bias_types), figsize=(10,14))

for i in range(len(models)):
    for j in range(len(bias_types)):
        
        
        row = df.loc[(df['model'] == models[i]) &(df['bias type'] == bias_types[j])]

        x = ['bias', 'key typo', 'middle random', 'letter swap']
        y = [row['bias mean'].item(), row['key typo mean'].item(), row['middle random mean'].item(), row['letter swap mean'].mean()]
        e = [row['bias std'].item(), row['key typo std'].item(), row['middle random std'].item(), row['letter swap std'].mean()]
        axs[i,j].errorbar(x, y, yerr=e, fmt='o')    
        axs[i,j].axhline(y = row['original mean'].item(), color = 'r', linestyle = '-') 
        axs[i,j].set_ylim(0,1)

        axs[i,j].set_xticklabels(clean_labels, rotation=90)

        axs[i,j].set_title(clean_bias_labels[j])
        if i != len(models)-1:
            axs[i,j].get_xaxis().set_visible(False)


grid = plt.GridSpec(len(models),len(bias_types))
for k in range(len(clean_model_labels)):
    create_subtitle(fig, grid[k, ::], clean_model_labels[k])
                
fig.tight_layout()                
plt.savefig("uncertainty.pdf", format="pdf", bbox_inches="tight")


In [ ]:
from scipy import stats

clean_model_labels = ['Llama2-7b', 'Llama2-13b', 'Llama2-70b', 'Solar', 'GPT 3.5 Turbo', 'GPT 3.5 Turbo Instruct']
models = ['llama2-7b', 'llama2-13b', 'llama2-70b', 'llama2-70b-ift', 'llama2-7b-chat', 'llama2-13b-chat', 'llama2-70b-chat', 'gpt-3.5-turbo', 'gpt-3.5-turbo-instruct']  #these dont have perturbations yet
bias_types = ['acquiescence','response_order', 'odd_even', 'opinion_float', 'allow_forbid']
clean_labels = ['Bias', 'Key Typo', 'Middle Random', 'Letter Swap']
clean_bias_labels = ['Acquiescence', 'Response Order', 'Odd/even', 'Opinion Float', 'Allow/forbid']


effect_sizes = []
uncertainties = []

for i in range(len(models)):
    
    for j in range(len(bias_types)):
                    
        values, pvals, keys = run_stat_test(models[i], bias_types[j])
        effect_sizes = {'keys': keys, 'effects': values} 
        df_effect = pd.DataFrame(effect_sizes)
        df_effect['effects'] = abs(df_effect['effects'])/100

    
        keys, values = get_indiv_entropies(models[i], bias_types[j])
        entps = {'keys': keys, 'entps': values} 
        df_entps = pd.DataFrame(entps)
        
        df_comb = pd.merge(df_effect, df_entps, on='keys')            
        stat_results = stats.pearsonr(df_comb['effects'], df_comb['entps'])
        #print(round(stat_results.statistic,2), round(stat_results.pvalue,2))